# 뉴스 감성 분석

In [82]:
# pip install transformers
# pip install tf-keras
# pip install vaderSentiment

In [83]:
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import tensorflow
import re

In [84]:
news_df = pd.read_csv("../데이터셋/ProQuest_Articles_Translated_2.csv")

In [85]:
news_df.drop('Source', axis=1, inplace=True)

In [86]:
# 날짜 추출 함수 (YYYY-MM-DD 형식 변환)
def extract_date(text):
    # 날짜 패턴 찾기 (다양한 형식 지원)
    match = re.search(r'(\d{4}[-/.]\d{2}[-/.]\d{2})|(\d{2}[thrdn]*\s\w+\s\d{4})', text)

    if match:
        date_str = match.group(0)

        # 날짜 형식을 YYYY-MM-DD로 변환
        try:
            parsed_date = pd.to_datetime(date_str, errors='coerce', dayfirst=False)
            return parsed_date.strftime('%Y-%m-%d')  # '0000-00-00' 형식
        except:
            return None  # 변환 실패 시 None 처리

    return None  # 날짜 패턴이 없는 경우

# 날짜 열 변환
news_df['Date'] = news_df['Date'].apply(extract_date)


In [87]:
news_df[news_df['Date'].isna()]

,Title,Date,Title_Translated
91,A moral and a practical failure Trump's withdr...,None,트럼프의 파리 협정 탈퇴로 인한 도덕적 및 실질적 실패는 글로벌 위기에서 미국의 역...
171,Trump's withdrawal from the Paris Climate Agre...,None,트럼프의 파리 기후 협정 탈퇴는 아프리카와 지구에 해를 끼칩니다!
179,Trump administration takes aim at EPA policy,None,"트럼프 행정부, EPA 정책을 겨냥하다"
188,EPA mum on Trump order's effect on East Chicag...,None,트럼프 명령이 동부 시카고 청소에 미치는 영향에 대한 EPA 엄마


In [88]:
# 날짜 None인 행 삭제
news_df.drop([91, 171, 179, 188], axis=0, inplace=True)

In [89]:
# 감성 분석 모델 로드 (Hugging Face 'DistilBERT' 기반)
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# 감성 점수 계산 함수
def get_bert_sentiment(text):
    if pd.isna(text) or text.strip() == "":
        return 0  # 빈 문장은 중립(0) 처리
    result = sentiment_pipeline(text[:512])  # BERT는 최대 512 토큰까지 처리 가능
    label = result[0]['label']
    score = result[0]['score']
    return score if label == "POSITIVE" else -score  # 긍정이면 양수, 부정이면 음수

# 뉴스 본문 감성 분석 적용
news_df['Sentiment_Score'] = news_df['Title'].apply(get_bert_sentiment)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Device set to use 0


In [90]:
news_df

,Title,Date,Title_Translated,Sentiment_Score
0,BACK STORY; Trump's old-energy plan nears a cr...,2018-01-11,배경 이야기; 트럼프의 오래된 에너지 계획은 중요한 시점에 가까워지고 있습니다; 그...,0.971675
1,Trump signs bill squashing Obama-era financial...,2017-02-14,"트럼프, 화석 연료에 대한 오바마 시대의 금융 규칙을 무너뜨리는 법안 서명",-0.990895
2,Trump Stresses Fossil Fuel Agenda In Energy Po...,2016-09-23,"트럼프, 에너지 정책 연설에서 화석 연료 의제 강조",-0.964925
3,Trump's energy policy perils; His embrace of f...,2016-11-22,"트럼프의 에너지 정책은 위험에 처해 있으며, 화석 연료에 대한 그의 포용과 오바마의...",0.986535
4,"Foolish & costly; Moves by universities, munic...",2017-06-29,어리석고 비용이 많이 듭니다; 특히 트럼프 대통령이 파리 기후 협정에서 미국의 탈퇴...,-0.998990
...,...,...,...,...
341,Calif. death reported in E. coli outbreak tied...,2018-05-03,캘리포니아에서 아리즈와 관련된 대장균 발병으로 사망한 사례가 보고되었습니다. 상추;...,-0.998271
342,Trump's Yemen policy survives Senate's veto ov...,2019-05-03,트럼프의 예멘 정책은 상원의 거부권 무효화 입찰에서 살아남았습니다; 세인트. 루시아...,-0.890819
343,Press Secretary Spicer's blunder hard to belie...,2017-04-13,스파이서 장관의 실수는 믿기 어렵습니다; 스파이서 장관은 트럼프와 달리 실수를 인정...,-0.975777
344,Trump's Yemen policy survives Senate's veto ov...,2019-05-03,트럼프의 예멘 정책은 상원의 거부권 무효화 입찰에서 살아남았습니다. 뉴스 브리핑; ...,-0.773825


In [93]:
news_df.to_csv('../데이터셋/news_sentiment_analysis.csv', index=False)